# Biometric Verification System for Secure Voting

This notebook demonstrates the implementation and usage of a multi-modal biometric verification system that combines:
1. Face Spoof Detection
2. Face Recognition
3. Iris Detection
4. Iris Recognition

The system is designed to provide maximum security for voting applications.

## Setup and Installation

First, let's mount Google Drive and install required dependencies.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Clone the repository
!git clone https://github.com/your-repo/biometric-verification.git
!cd biometric-verification

# Install dependencies
!pip install -r biometric_models/colab_requirements.txt

## Import Required Libraries

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2

from biometric_models.verify import BiometricVerifier
from biometric_models.config import CONFIG

## Initialize Biometric Verifier

In [ ]:
# Initialize the verifier
verifier = BiometricVerifier(CONFIG)

# Helper function to load and preprocess images
def load_image(path):
    image = Image.open(path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

## Test Face Spoof Detection

In [ ]:
def test_face_spoof(image_path):
    # Load and process image
    image = load_image(image_path)
    
    # Perform spoof detection
    result = verifier.verify_liveness(image)
    
    # Display results
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.imshow(image.squeeze().permute(1, 2, 0))
    plt.title('Input Image')
    
    plt.subplot(1, 2, 2)
    plt.imshow(result['attention_map'].squeeze(), cmap='jet')
    plt.title(f"Attention Map\nLiveness Score: {result['confidence']:.2f}")
    
    plt.show()
    
    print(f"Is Live: {result['is_live']}")
    print(f"Confidence: {result['confidence']:.2f}")

## Test Face Recognition

In [ ]:
def test_face_recognition(probe_path, gallery_paths):
    # Load probe image
    probe_image = load_image(probe_path)
    
    # Create gallery embeddings
    gallery_embeddings = {}
    for identity, path in gallery_paths.items():
        gallery_image = load_image(path)
        embedding = verifier.face_recognizer.extract_features(gallery_image)
        gallery_embeddings[identity] = {
            'face_embedding': embedding
        }
    
    # Compare with gallery
    probe_embedding = verifier.face_recognizer.extract_features(probe_image)
    result = verifier._verify_face_embedding(probe_embedding, gallery_embeddings)
    
    print(f"Best Match: {result['identity']}")
    print(f"Similarity: {result['similarity']:.2f}")
    print(f"Match: {result['match']}")

## Test Iris Detection

In [ ]:
def test_iris_detection(image_path):
    # Load and process image
    image = load_image(image_path)
    
    # Perform iris detection
    result = verifier.verify_iris_quality(image)
    
    # Display results
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.imshow(image.squeeze().permute(1, 2, 0))
    plt.title('Input Image')
    
    # Draw bounding box
    if result['bbox']:
        x1, y1, x2, y2 = result['bbox']
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                         fill=False, color='red', linewidth=2))
    
    plt.subplot(1, 2, 2)
    plt.imshow(result['mask'].squeeze(), cmap='gray')
    plt.title(f"Iris Mask\nQuality Score: {result['quality_score']:.2f}")
    
    plt.show()
    
    print(f"Is Valid: {result['is_valid']}")
    print(f"Quality Score: {result['quality_score']:.2f}")

## Complete Verification Pipeline

In [ ]:
def verify_identity(face_path, iris_path, reference_embeddings):
    # Load images
    face_image = load_image(face_path)
    iris_image = load_image(iris_path)
    
    # Perform complete verification
    result = verifier.verify_identity(face_image, iris_image, reference_embeddings)
    
    print("Verification Results:")
    print(f"Success: {result['success']}")
    if result['success']:
        print(f"Verified Identity: {result['verified_identity']}")
        print(f"Final Score: {result['final_score']:.2f}")
        print("\nStep-wise Results:")
        for step, details in result['steps'].items():
            print(f"\n{step.replace('_', ' ').title()}:")
            for key, value in details.items():
                if isinstance(value, float):
                    print(f"  {key}: {value:.2f}")
                else:
                    print(f"  {key}: {value}")
    else:
        print(f"Error: {result.get('error', 'Unknown error')}")
    
    print(f"\nTotal Processing Time: {result['total_time']:.2f} seconds")

## Example Usage

Now let's test the system with some example images.

In [ ]:
# Test paths
test_face = 'path/to/face/image.jpg'
test_iris = 'path/to/iris/image.jpg'

# Example gallery
gallery_paths = {
    'person1': 'path/to/person1/face.jpg',
    'person2': 'path/to/person2/face.jpg'
}

# Test individual components
print("Testing Face Spoof Detection...")
test_face_spoof(test_face)

print("\nTesting Face Recognition...")
test_face_recognition(test_face, gallery_paths)

print("\nTesting Iris Detection...")
test_iris_detection(test_iris)

# Test complete pipeline
print("\nTesting Complete Verification Pipeline...")
verify_identity(test_face, test_iris, reference_embeddings)